# Analysis

### Setup

In [ ]:
from env import Hangman, HangmanCheat
from network import NNAgent, Network

In [ ]:
policy_network = Network()
policy_network.load_weights('policy.h5') #input p
player = NNAgent(policy_network)
player.eval() #setting to evaluation state so the agent will not memorize play history.
player.reset_guessed()

In [ ]:
# test setup
env = Hangman('words/top_1000.txt' , max_lives = 8, verbose = True)

done = False
state = env.reset()

while not done :
    guess = player.select_action(state)
    print('Guessing', guess)
    state, _ , done , _ = env.step(guess)
    
player.reset_guessed()

In [ ]:
# test cheat setup
env = HangmanCheat('words/top_1000.txt' , max_lives = 8, verbose = True, reject_rate=0.2)

done = False
state = env.reset()

while not done :
    guess = player.select_action(state)
    print('Guessing', guess)
    state, _ , done , _ = env.step(guess)
    
player.reset_guessed()

## *Honest* case

Analyze the given policy in the case, where the environment (so the second player) is honest.

Derive how many words are guessed correctly using the top 1000 frequent words for different maximum number of guesses.

In [ ]:
from tqdm import tqdm

# results dictionary for max_lives in range 1-10
results = {}

for max_lives in range(1, 11):
    print(f"Testing with max_lives = {max_lives}")
    print("=====================================")

    # (re-)initialize the environment
    env = Hangman('words/top_1000.txt' , max_lives = max_lives, verbose = False)
    player.reset_guessed()

    # count the number of correct guesses
    correct = 0

    # loop through all the words
    for word in tqdm(env.words):
        # reset the environment
        state = env.reset(word)
        done = False
        # play the game
        while not done:
            guess = player.select_action(state)
            state, _ , done , _ = env.step(guess)
        # check if the word is guessed correctly
        if env.is_game_won():
            correct += 1
        player.reset_guessed()
    results[max_lives] = correct / len(env.words)

print(results)

## *Cheat* case

Analyze the given policy in the case, where the environment (so the second player) is cheating.

Keep the number of maximum guesses fixed at first. Derive how many words are guessed correctly using the top 1000 frequent words and vary the cheating factor.

In [ ]:
from tqdm import tqdm

# results dictionary for reject_rate in range 0-1 and max_lives in range 1-10
results = {}

for max_lives in range(1, 11):
    for reject_rate in range(0, 10):
        reject_rate /= 10
        print(f"Testing with max_lives = {max_lives} reject_rate = {reject_rate}")
        print("=====================================")

        # (re-)initialize the environment
        env = HangmanCheat('words/sample_1000.txt' , max_lives = max_lives, verbose = False, reject_rate=reject_rate)
        player.reset_guessed()

        # count the number of correct guesses
        correct = 0

        # loop through all the words
        for word in tqdm(env.words):
            # reset the environment
            state = env.reset(word)
            done = False
            # play the game
            while not done:
                guess = player.select_action(state)
                state, _ , done , _ = env.step(guess)
            # check if the word is guessed correctly
            if env.is_game_won():
                correct += 1
            player.reset_guessed()
        results[(max_lives, reject_rate)] = correct / len(env.words)

    print(results)

## Results

#### Top 1000 words

| max_lives | 0.0   | 0.1   | 0.2   | 0.3   | 0.4   | 0.5   | 0.6   | 0.7   | 0.8   | 0.9   | 1.0   |
|----------:|:-----:|:-----:|:-----:|:-----:|:-----:|:-----:|:-----:|:-----:|:-----:|:-----:|:-----:|
| 1         | 0.013 | 0.005 | 0.005 | 0.002 | 0.001 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 |
| 2         | 0.051 | 0.030 | 0.020 | 0.012 | 0.011 | 0.004 | 0.001 | 0.000 | 0.000 | 0.000 | 0.000 |
| 3         | 0.106 | 0.065 | 0.033 | 0.018 | 0.005 | 0.001 | 0.000 | 0.003 | 0.002 | 0.001 | 0.000 |
| 4         | 0.210 | 0.127 | 0.070 | 0.040 | 0.019 | 0.013 | 0.006 | 0.003 | 0.000 | 0.000 | 0.000 |
| 5         | 0.317 | 0.178 | 0.107 | 0.061 | 0.033 | 0.014 | 0.010 | 0.004 | 0.000 | 0.000 | 0.000 |
| 6         | 0.434 | 0.251 | 0.155 | 0.069 | 0.046 | 0.024 | 0.009 | 0.006 | 0.003 | 0.001 | 0.000 |
| 7         | 0.552 | 0.307 | 0.176 | 0.108 | 0.040 | 0.025 | 0.018 | 0.004 | 0.006 | 0.002 | 0.000 |
| 8         | 0.646 | 0.375 | 0.193 | 0.128 | 0.066 | 0.038 | 0.014 | 0.006 | 0.002 | 0.002 | 0.000 |
| 9         | 0.721 | 0.432 | 0.268 | 0.148 | 0.074 | 0.033 | 0.017 | 0.006 | 0.005 | 0.001 | 0.000 |
| 10        | 0.781 | 0.458 | 0.274 | 0.150 | 0.082 | 0.046 | 0.017 | 0.014 | 0.002 | 0.003 | 0.000 |

#### Sample 1000 words

| max_lives | 0.0   | 0.1   | 0.2   | 0.3   | 0.4   | 0.5   | 0.6   | 0.7   | 0.8   | 0.9   | 1.0   |
|----------:|:-----:|:-----:|:-----:|:-----:|:-----:|:-----:|:-----:|:-----:|:-----:|:-----:|:-----:|
| 1         | 0.020 | 0.008 | 0.002 | 0.000 | 0.001 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 |
| 2         | 0.067 | 0.030 | 0.015 | 0.006 | 0.002 | 0.001 | 0.000 | 0.000 | 0.001 | 0.000 | 0.000 |
| 3         | 0.148 | 0.079 | 0.032 | 0.022 | 0.004 | 0.004 | 0.000 | 0.000 | 0.000 | 0.000 | 0.000 |
| 4         | 0.239 | 0.113 | 0.057 | 0.022 | 0.009 | 0.005 | 0.000 | 0.002 | 0.000 | 0.000 | 0.000 |
| 5         | 0.360 | 0.188 | 0.090 | 0.051 | 0.018 | 0.008 | 0.001 | 0.002 | 0.001 | 0.000 | 0.000 |
| 6         | 0.493 | 0.268 | 0.128 | 0.066 | 0.025 | 0.015 | 0.007 | 0.003 | 0.000 | 0.000 | 0.000 |
| 7         | 0.581 | 0.291 | 0.152 | 0.066 | 0.045 | 0.017 | 0.006 | 0.001 | 0.000 | 0.000 | 0.000 |
| 8         | 0.659 | 0.338 | 0.197 | 0.080 | 0.035 | 0.017 | 0.006 | 0.003 | 0.000 | 0.000 | 0.000 |
| 9         | 0.730 | 0.377 | 0.200 | 0.099 | 0.043 | 0.020 | 0.005 | 0.001 | 0.000 | 0.001 | 0.000 |
| 10        | 0.777 | 0.414 | 0.226 | 0.094 | 0.051 | 0.025 | 0.009 | 0.005 | 0.000 | 0.001 | 0.000 |